Code heavily based on the Notebook from Week3 : https://colab.research.google.com/drive/1EyJZbbbjoSxHvqJtTxJBAtiP9S6CE7FL?authuser=1

### Importing the libraries

In [60]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from string import punctuation
import torch
import pandas as pd
from time import time
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
nltk.download("")

[nltk_data] Error loading : Package '' not found in index


False

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
tf.test.gpu_device_name()

''

### Loading the data

In [39]:
df = pd.read_csv("drive/MyDrive/Extramarks Project/Dataset versions/EM-Data-with-answers.csv", index_col = 0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [40]:
print("question_nans: ", df['question'].isna().sum())
print("answer_nans: ", df['answer'].isna().sum())
print("question_id_nans: ", df['question_id'].isna().sum())
print("syllabus_id_nans: ", df['syllabus_id'].isna().sum())
print("question_type_nans: ", df['question_type'].isna().sum())

question_nans:  496
answer_nans:  1688
question_id_nans:  0
syllabus_id_nans:  0
question_type_nans:  0


We are intentionally not deleting the values where answer is NaN becasue we ight use SERP api for those questions

In [41]:
len1 =len(df)
df = df[df['question'].notna()]
print("Rows with NaN values: ", len1-len(df))

Rows with NaN values:  496


In [38]:
# len1 =len(df)
# df = df.dropna()
# print("Rows with NaN values: ", len1-len(df))

Rows with NaN values:  1846


In [42]:
df.head(2)

,question_id,question,syllabus_id,answer,question_type
0,2073394,Total number of species that can be oxidized b...,698013,All the species which are not in their highest...,Subjective Question
1,2073385,How many following Ammonium salts will evolve ...,698013,Ammonium salts having anions NO2 NO3 ClO4 and ...,Subjective Question


In [43]:
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])
print(df.sample(1).iloc[0]['question'])

न म नल ख त म स एक ल क क त ह 
Choose the correct option 
What are demonstrative pronouns 
_Fill in the blanks _ 1 The digital signal to analog signals are carried out by _____________ so that data can travel over telephone lines 2 _____________ cable is a kind of cable made by putting two separate wires together 3 The information that remains same for each version of the mail merged document in MS Word is known as______________ 4 The ____________ feature allows you to highlight and view changes made to a document 5 _____________ provides communication service upto 100 km 
क र य क अर थ ह 


######Few samples

In [44]:
df.iloc[-200:-190]

,question_id,question,syllabus_id,answer,question_type
275861,414978,The adverb indispensably means _______________,3623,importantly,Single Choice Question
275862,414976,Birds fly ________ fish swim,3632,and,Single Choice Question
275863,414975,Choose the appropriate conjunction You will pa...,3632,if,Single Choice Question
275864,414974,Choose the appropriate conjunction Take care o...,3632,before,Single Choice Question
275865,414933,Choose the appropriate conjunction Answer the ...,3632,before,Single Choice Question
275866,414932,Choose the appropriate conjunction I returned ...,3632,after,Single Choice Question
275867,414931,Choose the appropriate conjunction Many things...,3632,since,Single Choice Question
275868,414930,Choose the appropriate conjunction I am so tir...,3632,that,Single Choice Question
275869,414929,Choose the appropriate conjunction Don t waste...,3632,lest,Single Choice Question
275870,414684,Choose the appropriate conjunction My brother ...,3632,while,Single Choice Question


In [45]:
df.iloc[-300:-290]

,question_id,question,syllabus_id,answer,question_type
275761,416271,The population of China is greater ___________...,4415,than that of,Single Choice Question
275762,416270,Either you or I ___________________,4415,am mistaken,Single Choice Question
275763,416269,The collective noun choir means ___________,3637,a group of singers,Single Choice Question
275764,416268,The proper noun in the sentence is ___________...,3637,Kanpur,Single Choice Question
275765,416267,The correct collective noun in the following i...,3637,a pride of peacocks,Single Choice Question
275766,416266,John has hatred for swimming The abstract noun...,3637,hatred,Single Choice Question
275767,416265,The correct adjective order is _______________,3620,a hardworking short forty year old Punjabi dri...,Single Choice Question
275768,416264,We are a good team as ____________________ our...,3636,each of,Single Choice Question
275769,416263,Choose the correctly punctuated sentence,3616,Come here Tom and sit down,Single Choice Question
275770,416262,Choose the correct use of capital letters,3616,The Oxford English Dictionary is published by ...,Single Choice Question


In [46]:
df.iloc[-500:-490]

,question_id,question,syllabus_id,answer,question_type
275561,418572,The rules made for the Buddhist Sangha were wr...,744376,Vinaya Pitaka,Single Choice Question
275562,418571,Buddha's main teaching was based on the fact t...,744376,Life is full of sufferings and unhappiness,Single Choice Question
275563,418570,The founder of Buddhism was,744376,Siddartha,Single Choice Question
275564,418569,The patient had died before ____________,3659,the doctor came,Single Choice Question
275565,418568,Amy was studying while Mollie _______________ ...,3659,was making,Single Choice Question
275566,418567,Last night at 6 PM __________________ dinner,3659,I was eating,Single Choice Question
275567,418566,Last year I,3659,took my exam,Single Choice Question
275568,418565,Isabel rarely ____________ to attend the music...,3659,comes,Single Choice Question
275569,418544,Yesterday my mother and I ______________ to wa...,3659,went,Single Choice Question
275570,418541,My aunt ______________ to my house occasionally,3659,comes,Single Choice Question


### Cleaning the data

In [47]:
removed_questions = []
questions_left = []

##### If there is a number in the question delete it 

In [48]:
for i in range(len(df)):
  if i % 20000 == 0:
    print("Index: ", i)
  patternDigit = re.compile(r'.*\d.*')
  row = df.iloc[i]
  question = row['question']
  # try:
  if len(patternDigit.findall(question)) > 0:
    removed_questions.append(row)
  else:
    questions_left.append(row)
  # except:
  #   print("Error at index: ", str(i), ", type: ", str(type(question)), ", question: " , str(question), ", row: " +str(row))
  #   removed_questions.append(row) 

print(len(removed_questions))
print(len(questions_left))

Index:  0
Index:  20000
Index:  40000
Index:  60000
Index:  80000
Index:  100000
Index:  120000
Index:  140000
Index:  160000
Index:  180000
Index:  200000
Index:  220000
Index:  240000
Index:  260000
117174
158392


Hence we remove 42% of the questions

##### If there are more than 3 numbers concsecutively in the question delete it 

In [49]:
# removed_questions = []
# questions_left = []

In [50]:
# for i in range(len(df[:100000])):
#   if i % 20000 == 0:
#     print("Index: ", i)
#   patternDigit = re.compile(r'.*\d\d\d.*')
#   row = df.iloc[i]
#   question = row['question']
#   # try:
#   if len(patternDigit.findall(question)) > 0:
#     removed_questions.append(row)
#   else:
#     questions_left.append(row)
#   # except:
#   #   print("Error at index: ", str(i), ", type: ", str(type(question)), ", question: " , str(question), ", row: " +str(row))
#   #   removed_questions.append(row) 

# print(len(removed_questions))
# print(len(questions_left))

In [51]:
# temp = []
# for i in range(len(df[:100000])):
#   if i % 20000 == 0:
#     print("Index: ", i)
#   patternDigit = re.compile(r'.*\d.*')
#   row = df.iloc[i]
#   question = row['question']
#   # try:
#   if len(patternDigit.findall(question)) > 0:
#     temp.append(row)
#   # except:
#   #   print("Error at index: ", str(i), ", type: ", str(type(question)), ", question: " , str(question), ", row: " +str(row))
#   #   removed_questions.append(row) 

# print(len(temp))

In [52]:
# pd.DataFrame(temp)

Hence we remove 20% of the questions

##### If there is a non-ASCII charater in the question delete it 
The devanagri chars have the unicod eindexes 0900 - 097F according to [stackoverflow](https://stackoverflow.com/questions/41356013/how-to-detect-if-a-string-contains-hindi-devnagri-in-it-with-character-and-wor)

In [53]:
cleanDf = pd.DataFrame(questions_left)
questions_left = []

In [54]:
for i in range(len(cleanDf)):
  if i % 20000 == 0:
    print("Index: ", i)
  patternHindi = re.compile(r'.*[\u0900-\u097F].*') 
  row = cleanDf.iloc[i]
  question = row['question']
  
  if len(patternHindi.findall(question)) > 0:
    removed_questions.append(row)
  else:
    questions_left.append(row)
  

print(len(removed_questions))
print(len(questions_left))

Index:  0
Index:  20000
Index:  40000
Index:  60000
Index:  80000
Index:  100000
Index:  120000
Index:  140000
141135
134431


##### Replacing all element shortcuts with their full forms

In [55]:
# cleanDf = pd.DataFrame(questions_left)
# questions_left = []

In [57]:
# zync potassium 

### Saving as file

In [58]:
cleanDf.to_csv("drive/MyDrive/Extramarks Project/Dataset versions/JEE-with-answers-without-numericals.csv")

In [59]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("DONE. GMT(00:00) time is :", current_time)

Current Time is : 10:34:30
